In [2]:
'''4 Метод обратной матрицы, Метод Гаусса с выбором главного элемента по столбцу,
метод итерации'''
'''a,b,c,d = 1,3,0,9'''

'''
(1+a)x1 + 14х2 - 15х3 + 23х4 = 5 
16x1 + (1+b)х2 - 22х3 + 29х4 = 8 
18x1 + 20х2 – (1+c)х3 + 32х4 = 9 
10x1 + 12х2 - 16х3 + (1+d)х4 = 4.
'''


import numpy as np

a,b,c,d = 1.,3.,0.,9.


M1 = np.array([[1.+a, 14., -15., 23.], [16., 1.+b, -22., 29.], [18., 20., -(1.+c), 32.], [10., 12., -16., 1.+d]])
v1 = np.array([5.,8.,9.,4.])
M1, v1

(array([[  2.,  14., -15.,  23.],
        [ 16.,   4., -22.,  29.],
        [ 18.,  20.,  -1.,  32.],
        [ 10.,  12., -16.,  10.]]), array([5., 8., 9., 4.]))

In [3]:
def GEPPsolve(a, b):
    n = len(a)

    # Elimination
    for k in range(n-1):
        # Pivot
        maxindex = abs(a[k:,k]).argmax() + k
        if a[maxindex, k] == 0:
            raise ValueError("Matrix is singular.")
        # Swap
        if maxindex != k:
            a[[k,maxindex]] = a[[maxindex, k]]
            b[[k,maxindex]] = b[[maxindex, k]]
        # Eliminate
        for row in range(k+1, n):
            multiplier = a[row,k]/a[k,k]
            a[row, k:] = a[row, k:] - multiplier * a[k, k:]
            b[row] = b[row] - multiplier * b[k]
    # Back Substitution
    x = np.zeros(n)
    for k in range(n-1, -1, -1):
        x[k] = (b[k] - a[k,k+1:].dot(x[k+1:])) / a[k, k]
    return x


def ITERsolve(a, b, iter_max):
    x = np.zeros_like(b)
    for it_count in range(iter_max):
        x_new = np.zeros_like(x)

        for i in range(a.shape[0]):
            s1 = np.dot(a[i, :i], x[:i])
            s2 = np.dot(a[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / a[i, i]

        if np.allclose(x, x_new, atol=1e-10):
            break
        x = x_new
    return x

In [19]:
def Obr(a, b):
    if np.linalg.det(a):
        aobr = np.linalg.inv(a)
        return aobr.dot(b).T
    return None

In [21]:
Obr(M1, v1)

array([ 0.15752769,  0.03555312, -0.01873376,  0.16983454])

In [8]:
GEPPsolve(M1, v1)

array([ 0.15752769,  0.03555312, -0.01873376,  0.16983454])

In [31]:
ITERsolve(M1, v1, 10000000)

array([ 0.15752769,  0.03555312, -0.01873376,  0.16983454])

In [33]:
np.linalg.eigh(M1)

(array([-32.93548387, -17.90646425, -13.77777778,  18.        ]),
 array([[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]))

In [22]:
np.linalg.det(M1)

-146260.00000000006

In [25]:
np.linalg.norm(M1)

64.70690126858845

In [24]:
M1_int = np.array([[int(j) for j in i]for i in M1])
v1_int = np.array([[int(i) for i in v1]])

In [26]:
np.concatenate((M1_int, v1_int))

array([[ 18,  20,  -1,  32],
       [  0, -13, -21,   0],
       [  0,   0, -32,  19],
       [  0,   0,   0, -17],
       [  9,   0,   4,  -3]])

In [27]:
np.linalg.matrix_rank(M1), np.linalg.matrix_rank(np.concatenate((M1_int, v1_int)))

(4, 4)

In [28]:
np.linalg.solve(M1, v1)

array([ 0.15752769,  0.03555312, -0.01873376,  0.16983454])

In [29]:
def calc_err(sol, a, b):
    return a.dot(sol) - b

In [30]:
import time
from scipy import linalg
from numpy import dot


print (np.linalg.eigh(M1)[0])

rep_times = 50000

t1 = time.clock()
for _ in range(rep_times):
    ge_builtin_solution = np.linalg.solve(M1, v1)
ge_builtin_time = (time.clock() - t1) * 1000.0**2 / rep_times

t1 = time.clock()
for _ in range(rep_times):
    ge_solution = GEPPsolve(M1.copy(), v1.copy())
ge_time = (time.clock() - t1) * 1000.0**2 / rep_times

t1 = time.clock()
for _ in range(rep_times):
    iter_solution = ITERsolve(M1, v1, 10000)
iter_time = (time.clock() - t1) * 1000.0**2 / rep_times

t1 = time.clock()
for _ in range(rep_times):
    obr_sol = Obr(M1, v1)
obr_time = (time.clock() - t1) * 1000.0**2 / rep_times

print ("ge-builtin       : %s error: %s %s mcs" % (ge_builtin_solution, calc_err(ge_builtin_solution, M1, v1), ge_builtin_time))
print ("g-elimination    : %s error: %s %s mcs" % (ge_solution, calc_err(ge_solution, M1, v1), ge_time))
print ("iter_sol         : %s error: %s %s mcs" % (iter_solution, calc_err(iter_solution, M1, v1), iter_time))
print ("obr_sol          : %s error: %s %s mcs" % (obr_sol, calc_err(obr_sol, M1, v1), obr_time))

[-32.93548387 -17.90646425 -13.77777778  18.        ]
ge-builtin       : [ 0.15752769  0.03555312 -0.01873376  0.16983454] error: [0. 0. 0. 0.] 10.951839999999988 mcs
g-elimination    : [ 0.15752769  0.03555312 -0.01873376  0.16983454] error: [0. 0. 0. 0.] 45.17822000000001 mcs
iter_sol         : [ 0.15752769  0.03555312 -0.01873376  0.16983454] error: [0. 0. 0. 0.] 278.60938 mcs
obr_sol          : [ 0.15752769  0.03555312 -0.01873376  0.16983454] error: [0.00000000e+00 2.22044605e-16 0.00000000e+00 0.00000000e+00] 117.84942 mcs
